## **RQ1**

> *Do N-HiTS and N-BEATS maintain competitive performance across different atmospheric pollutants (PM2.5, PM10, O₃, NO₂) and forecast horizons?*
>> *Do global neural time series models (N-HiTS and N-BEATS) maintain competitive and robust performance across different atmospheric pollutants (PM2.5, PM10, O₃, NO₂) and forecast horizons (1–30 days), when compared to naïve, classical statistical, and simple global baselines, under both average and extreme-oriented evaluation metrics?*

* Evaluation across multiple horizons:

  * short-term (1–7 days),
  * medium-term (14 and 30 days).
* Comparison against naïve, statistical, and simple global forecasting baselines.
* Use of both average performance metrics and metrics explicitly oriented toward extreme events.

## Libraries

In [32]:
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

import optuna
import itertools
import shutil
import time
import functools
import gc

import pandas as pd
import numpy as np
np.random.seed(1)

import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import plotly.io as pio
from graphmodex import plotlymodex
pio.renderers.default = 'notebook'

import joblib
import pickle
from IPython.display import clear_output

In [42]:
import neuralforecast
import mlforecast
import statsforecast
import utilsforecast
import coreforecast

from statsforecast import StatsForecast
from statsforecast.models import (
    Naive, SeasonalNaive, 
    AutoARIMA, AutoCES, AutoETS, AutoTheta,
)

from mlforecast import MLForecast
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.losses.pytorch import MSE, SMAPE, MAE

from mlforecast.utils import PredictionIntervals

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")


# ==================================================
# REPRODUCTIBILITY
# ==================================================
import random
import torch

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


In [34]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="overflow encountered in square",
    category=RuntimeWarning,
)

### Results Storage

In [35]:
from pathlib import Path

BASE_RESULTS = Path("Results/RQ1")
BASE_RESULTS.mkdir(parents=True, exist_ok=True)

def save_results(df, model_family, pollutant, horizon_label):
    out_dir = BASE_RESULTS / model_family
    out_dir.mkdir(parents=True, exist_ok=True)

    fname = f"{pollutant}_{horizon_label}.csv"
    df.to_csv(out_dir / fname, index=False)

BASE_DIR_Pilot = Path("Results/RQ1/Pilot")
(BASE_DIR_Pilot / "studies").mkdir(parents=True, exist_ok=True)
(BASE_DIR_Pilot / "tables").mkdir(parents=True, exist_ok=True)
(BASE_DIR_Pilot / "meta").mkdir(parents=True, exist_ok=True)

## Data & Ablation

In [29]:
# ===============================
# DATA
# ===============================
df = pd.read_parquet(r'..\Data\CAMS\processed\eac4_era5_2010_2024_brasil_enhanced.parquet')

pm10 = (
    df
    .rename(columns={
        'pm10': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

pm2p5 = (
    df
    .rename(columns={
        'pm2p5': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

go3 = (
    df
    .rename(columns={
        'go3': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

no2 = (
    df
    .rename(columns={
        'no2': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

In [31]:
# go3.to_parquet(r'.\Results\RQ1\full\go3.parquet')
# pm10.to_parquet(r'.\Results\RQ1\full\nopm102.parquet')
# pm2p5.to_parquet(r'.\Results\RQ1\full\nopm2p52.parquet')
# no2.to_parquet(r'.\Results\RQ1\full\no2.parquet')

### Setup

In [12]:
# ===============================
# PILOT CONFIG
# ===============================
POLLUTANT_abl = 'pm10'
UID_abl = 0

steps_per_day = 8
horizons = {
    '1d': 1 * steps_per_day,
    '7d': 7 * steps_per_day,
    '14d': 14 * steps_per_day,
    '30d': 30 * steps_per_day,
}

# ===============================
# DATA
# ===============================
df_abl = (
    df
    .query("unique_id == @UID_abl")
    .rename(columns={'pm10': 'y', 'valid_time': 'ds'})
    [['unique_id', 'ds', 'y']]
    .sort_values('ds')
)

In [13]:
# ===============================
# CONFIG
# ===============================
steps_per_day = 8
two_year_steps = 2 * 365 * steps_per_day
target_windows = 30

FREQ = '3h'
SEASON_LENGTH = 8 

pollutants_dict = {
        'go3': {
            'df': go3,
            'scaler': 1e8
        },
        'no2': {
            'df': no2,
            'scaler': 1e10
        },
        'pm10': {
            'df': pm10,
            'scaler': 1e9
        },
        'pm2p5': {
            'df': pm2p5,
            'scaler': 1e9
        },
    }
experiments_dict = {
    '1 days': {
        'horizon': 8*1,
        'step_size': max(8*1, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '7 days': {
        'horizon': 8*7,
        'step_size': max(8*7, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '14 days': {
        'horizon': 8*14,
        'step_size': max(8*14, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '30 days': {
        'horizon': 8*30,
        'step_size': 8*30,
        'windows': two_year_steps // (8*30),  # 24
    },
}

## **Models**

### Pilot for Ablation

In [ ]:
# =========================================================
# PILOT STUDY — MULTI-OBJECTIVE (MAE + MAE_p95)
# NHITS | 1 cell | 1 pollutant | multiple horizons
# =========================================================

# ---------------------------------------------------------
# HARDENING (CPU / Windows stability)
# ---------------------------------------------------------
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)

# ---------------------------------------------------------
# REPRODUCIBILITY
# ---------------------------------------------------------
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ---------------------------------------------------------
# ASSUMED EXISTING OBJECTS
# ---------------------------------------------------------
# df_abl   : DataFrame with columns ['unique_id', 'ds', 'y']
#            containing ONLY ONE unique_id
# horizons : dict, e.g. {'1d':8,'7d':56,'14d':112,'30d':240}
# FREQ     : '3h'
# ---------------------------------------------------------

# ---------------------------------------------------------
# SAFE MAE_p95
# ---------------------------------------------------------
def mae_p95_safe(
    y: np.ndarray,
    yhat: np.ndarray,
    min_points: int = 10
) -> float:
    p95 = np.percentile(y, 95)
    mask = y >= p95
    if mask.sum() < min_points:
        return np.inf
    return float(np.mean(np.abs(y[mask] - yhat[mask])))

# ---------------------------------------------------------
# OPTUNA OBJECTIVE (MULTI-OBJECTIVE)
# ---------------------------------------------------------
def objective(trial, h: int):

    input_multiplier = trial.suggest_categorical(
        "input_multiplier", [2, 4, 6]
    )
    input_size = int(input_multiplier * h)

    model = NHITS(
        h=h,
        input_size=input_size,
        n_blocks=[1, 1, 1],
        mlp_units=3 * [[256, 256]],
        n_pool_kernel_size=[2, 2, 1],
        n_freq_downsample=[4, 2, 1],
        activation="ReLU",
        dropout_prob_theta=0.1,

        # Pilot setup
        max_steps=200,
        learning_rate=1e-3,
        batch_size=32,
        windows_batch_size=64,
        random_seed=SEED,
        loss=MAE(),
        logger=False,
    )

    nf = NeuralForecast(models=[model], freq=FREQ)

    cv = nf.cross_validation(
        df=df_abl,
        h=h,
        n_windows=10,
        step_size=h,
        refit=True,
    )

    y = cv["y"].to_numpy()
    yhat = cv["NHITS"].to_numpy()

    mae = float(np.mean(np.abs(y - yhat)))
    mae_p95 = mae_p95_safe(y, yhat, min_points=10)

    del nf, model, cv
    gc.collect()

    # MULTI-OBJECTIVE RETURN
    return mae, mae_p95

# ---------------------------------------------------------
# RUN PILOT
# ---------------------------------------------------------
pilot_records = []
pareto_summary = []

for label, h in horizons.items():

    print(f"\n🔎 Horizon: {label} ({h} steps)")

    sampler = optuna.samplers.TPESampler(seed=SEED)
    study = optuna.create_study(
        directions=["minimize", "minimize"],
        sampler=sampler,
    )

    study.optimize(
        lambda t: objective(t, h),
        n_trials=20,
        gc_after_trial=True,
    )

    study_path = BASE_DIR_Pilot / "studies" / f"study_horizon_{label}.pkl"

    with open(study_path, "wb") as f:
        pickle.dump(study, f)

    # Store all trials
    for t in study.trials:
        pilot_records.append({
            "horizon": label,
            "h": h,
            "input_multiplier": t.params.get("input_multiplier"),
            "MAE": t.values[0],
            "MAE_p95": t.values[1],
            "state": str(t.state),
        })

    # Store Pareto front
    for t in study.best_trials:
        pareto_summary.append({
            "horizon": label,
            "h": h,
            "input_multiplier": t.params["input_multiplier"],
            "MAE": t.values[0],
            "MAE_p95": t.values[1],
        })

# ---------------------------------------------------------
# RESULTS
# ---------------------------------------------------------
pilot_df = pd.DataFrame(pilot_records)
pareto_df = pd.DataFrame(pareto_summary)

print("\n=== PARETO FRONT (PER HORIZON) ===")
print(pareto_df.sort_values(["horizon", "MAE_p95", "MAE"]))

pilot_df_path = BASE_DIR_Pilot / "tables" / "pilot_trials.pkl"
pareto_df_path = BASE_DIR_Pilot / "tables" / "pilot_pareto.pkl"

pilot_df.to_pickle(pilot_df_path)
pareto_df.to_pickle(pareto_df_path)

pilot_df.to_csv(BASE_DIR_Pilot / "tables" / "pilot_trials.csv", index=False)
pareto_df.to_csv(BASE_DIR_Pilot / "tables" / "pilot_pareto.csv", index=False)

meta = {
    "seed": SEED,
    "freq": FREQ,
    "horizons": horizons,
    "n_trials": 20,
    "model": "NHITS",
    "loss": "MAE + MAE_p95",
    "date": pd.Timestamp.now().isoformat(),
}

with open(BASE_DIR_Pilot / "meta" / "experiment_meta.pkl", "wb") as f:
    pickle.dump(meta, f)

### Neural Forecasters

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():

        clear_output(wait=True)
        print(f"Running NEURAL | {pollutant_name} | {horizon_label}")

        # -------------------------------
        # MODELS
        # -------------------------------
        models = [
            NBEATS(
                h=experiment_['horizon'],
                input_size=4 * experiment_['horizon'],
                stack_types=["identity", "trend", "seasonality"],
                n_blocks=[1, 1, 1],
                mlp_units=3 * [[256, 256]],
                basis='polynomial',
                n_basis=2,
                n_harmonics=2,
                shared_weights=True,
                activation='ReLU',
                max_steps=500,
                learning_rate=1e-3,
                batch_size=32,
                windows_batch_size=1024,
                random_seed=SEED,
                alias='NBEATS-I',
                loss=MAE(),
            ),
            NBEATS(
                h=experiment_['horizon'],
                input_size=4 * experiment_['horizon'],
                stack_types=['identity'] * 3,
                n_blocks=[2, 2, 2],
                mlp_units=3 * [[256, 256]],
                shared_weights=False,
                activation='ReLU',
                max_steps=500,
                learning_rate=1e-3,
                batch_size=32,
                windows_batch_size=1024,
                random_seed=SEED,
                alias='NBEATS-G',
                logger=False,
                loss=MAE(),
            ),
            NHITS(
                h=experiment_['horizon'],
                input_size=4 * experiment_['horizon'],
                n_blocks=[1, 1, 1],
                mlp_units=3 * [[256, 256]],
                n_pool_kernel_size=[2, 2, 1],
                n_freq_downsample=[4, 2, 1],
                activation='ReLU',
                dropout_prob_theta=0.1,
                max_steps=500,
                learning_rate=1e-3,
                batch_size=32,
                windows_batch_size=1024,
                random_seed=SEED,
                alias='NHITS',
                logger=False,
                loss=MAE(),
            ),
        ]

        nf = NeuralForecast(models=models, freq=FREQ)

        # -------------------------------
        # DATA
        # -------------------------------
        df_ = pollutant_['df'].copy()
        df_.y = df_.y * pollutant_['scaler']

        start_time = time.time()
        results_nf = nf.cross_validation(
            df=df_,
            h=experiment_['horizon'],
            n_windows=experiment_['windows'],
            step_size=experiment_['step_size'],
            refit=True,
        )
        end_time = time.time()
        
        total_time = end_time - start_time
        time_per_window = total_time / experiment_['windows']
        results_nf['fit_time_seconds'] = total_time

        # -------------------------------
        # SCALE BACK
        # -------------------------------
        scale = pollutant_['scaler']
        for col in ['y', 'NHITS', 'NBEATS-G', 'NBEATS-I']:
            results_nf[col] = results_nf[col] / scale

        # -------------------------------
        # SAVE
        # -------------------------------
        save_results(
            results_nf,
            model_family="neural",
            pollutant=pollutant_name,
            horizon_label=horizon_label.replace(" ", "")
        )

### Statistical Forecasters

In [11]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():

        print(f"Running STATS | {pollutant_name} | {horizon_label}")

        # -------------------------------
        # MODELS
        # -------------------------------
        stat_models = [
            AutoARIMA(season_length=SEASON_LENGTH, alias='Arima'),
            AutoETS(season_length=SEASON_LENGTH, alias='ETS'),
            AutoTheta(season_length=SEASON_LENGTH, alias='Theta'),
            SeasonalNaive(season_length=SEASON_LENGTH, alias='SeasonalNaive-8'),
            SeasonalNaive(season_length=56, alias='SeasonalNaive-56'),
            Naive(alias='Naive'),
        ]

        sf = StatsForecast(
            models=stat_models,
            freq=FREQ,
            n_jobs=1,
        )

        # -------------------------------
        # DATA
        # -------------------------------
        df_ = pollutant_['df'].copy()

        # -------------------------------
        # CROSS-VALIDATION
        # -------------------------------
        start_time = time.time()
        results_sf = sf.cross_validation(
            df=df_,
            h=experiment_['horizon'],
            n_windows=experiment_['windows'],
            step_size=experiment_['step_size'],
            input_size=1*365*8, # 1 year
            refit=True,
        )
        end_time = time.time()
        
        total_time = end_time - start_time
        time_per_window = total_time / experiment_['windows']
        results_sf['fit_time_seconds'] = total_time
        
        # -------------------------------
        # SAVE
        # -------------------------------
        save_results(
            results_sf,
            model_family="stats",
            pollutant=pollutant_name,
            horizon_label=horizon_label.replace(" ", "")
        )

Running STATS | go3 | 1 days
Running STATS | go3 | 7 days
Running STATS | go3 | 14 days
Running STATS | go3 | 30 days
Running STATS | no2 | 1 days
Running STATS | no2 | 7 days
Running STATS | no2 | 14 days
Running STATS | no2 | 30 days
Running STATS | pm10 | 1 days
Running STATS | pm10 | 7 days
Running STATS | pm10 | 14 days
Running STATS | pm10 | 30 days
Running STATS | pm2p5 | 1 days
Running STATS | pm2p5 | 7 days
Running STATS | pm2p5 | 14 days
Running STATS | pm2p5 | 30 days


### Machine Learners

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():

        clear_output(wait=True)
        print(f"Running ML | {pollutant_name} | {horizon_label}")

        # -------------------------------
        # MODEL 
        # -------------------------------
        lgb_model = lgb.LGBMRegressor(
            n_estimators=500,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=SEED,
            n_jobs=-1,
        )
        rf_model = RandomForestRegressor(
            n_estimators=500,
            max_depth=10,
            random_state=SEED,
            n_jobs=-1,
        )
        xgb_model = xgb.XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=SEED,
            n_jobs=-1,
        )

        # -------------------------------
        # MLForecast (lags + calendar)
        # -------------------------------
        mlf = MLForecast(
            models={
                'LightGBM': lgb_model,
                'RandomForest': rf_model,
                'XGBoost': xgb_model,
            },
            freq=FREQ,
            lags=[1, 2, 4, 8, 16, 24, 56, 168, 336],
            lag_transforms={
                8:  [RollingMean(window_size=8), ExpandingMean()],
                56: [RollingMean(window_size=56)],
            },
            date_features=[
                'hour',       # ciclo intradiário (3h)
                'dayofweek',  # ciclo semanal
                'month',      # sazonalidade anual
                'dayofyear',  # sazonalidade anual mais “contínua”
            ],
        )

        # -------------------------------
        # DATA
        # -------------------------------
        df_ = pollutant_['df'].copy()
        df_ = df_.sort_values(['unique_id', 'ds'])

        # (opcional, mas recomendado) garantir dtype datetime
        df_['ds'] = pd.to_datetime(df_['ds'])

        # -------------------------------
        # CROSS-VALIDATION
        # -------------------------------
        start_time = time.time()
        results_ml = mlf.cross_validation(
            df=df_,
            h=experiment_['horizon'],
            n_windows=experiment_['windows'],
            step_size=experiment_['step_size'],
            refit=True,
        )
        end_time = time.time()

        total_time = end_time - start_time
        results_ml['fit_time_seconds'] = total_time

        # -------------------------------
        # SAVE
        # -------------------------------
        save_results(
            results_ml,
            model_family="ml",
            pollutant=pollutant_name,
            horizon_label=horizon_label.replace(" ", "")
        )

## Merge

In [40]:
FULL_DIR = BASE_RESULTS / "full"
FULL_DIR.mkdir(parents=True, exist_ok=True)

def build_full_results(pollutant, horizon_label):
    """
    Concatena neural + stats + ml para um (pollutant, horizon)
    """
    dfs = []

    for family in ["neural", "stats", "ml"]:
        fpath = BASE_RESULTS / family / f"{pollutant}_{horizon_label}.csv"

        if fpath.exists():
            df = pd.read_csv(fpath)

            # Remove a coluna 'fit_time' se ela existir, pois ela não deve ser parte do merge
            if 'fit_time_seconds' in df.columns:
                df = df.drop(columns=['fit_time_seconds'])
            
            dfs.append(df)
        else:
            print(f"⚠️ Missing: {fpath}")

    if not dfs:
        raise ValueError("No result files found.")

    # Merge progressivo (chaves comuns)
    df_full = dfs[0]
    for df in dfs[1:]:
        df_full = df_full.merge(
            df,
            on=["unique_id", "ds", "cutoff", "y"],
            how="inner"
        )

    return df_full

for pollutant_name in pollutants_dict.keys():
    for horizon_label in experiments_dict.keys():

        horizon_clean = horizon_label.replace(" ", "")

        print(f"Building FULL | {pollutant_name} | {horizon_clean}")

        df_full = build_full_results(
            pollutant=pollutant_name,
            horizon_label=horizon_clean
        )

        df_full.to_csv(
            FULL_DIR / f"{pollutant_name}_{horizon_clean}.csv",
            index=False
        )

Building FULL | go3 | 1days
Building FULL | go3 | 7days
Building FULL | go3 | 14days
Building FULL | go3 | 30days
Building FULL | no2 | 1days
Building FULL | no2 | 7days
Building FULL | no2 | 14days
Building FULL | no2 | 30days
Building FULL | pm10 | 1days
Building FULL | pm10 | 7days
Building FULL | pm10 | 14days
Building FULL | pm10 | 30days
Building FULL | pm2p5 | 1days
Building FULL | pm2p5 | 7days
Building FULL | pm2p5 | 14days
Building FULL | pm2p5 | 30days


## **Visualization**

In [ ]:
# ==================================================
# CONFIG
# ==================================================
uid = 0
pollutant_ = pollutants_dict['go3']
experiment_ = experiments_dict['7 days']

models = {
    'NHITS': "#287eb8",
    'NBEATS-G': "#72a549",
    'NBEATS-I': "#d4bc34",
    'LightGBM': "#6c34d4",
    'Naive': "#d43f34",
    'SeasonalNaive': "#d4347f",
}

cutoffs = results_ml['cutoff'].unique()

fig = go.Figure()

# ==================================================
# REAL SERIE
# ==================================================
pollutant_filt = pollutant_['df'].copy().query(
    "ds >= '2022-06-01' and unique_id == @uid"
)

fig.add_trace(
    go.Scatter(
        x=pollutant_filt['ds'],
        y=pollutant_filt['y'],
        mode='lines',
        line=dict(color='black', width=2),
        name='Real'
    )
)

# ==================================================
# MODELS — ONE WINDOW PER CUTOFF
# ==================================================
for model, color in models.items():
    show_legend_ = True
    for cutoff in cutoffs:

        df_fold = results_ml.query(
            "unique_id == @uid and cutoff == @cutoff"
        )

        if df_fold.empty:
            continue

        # Linha da previsão (segmento independente)
        fig.add_trace(
            go.Scatter(
                x=df_fold['ds'],
                y=df_fold[model],
                mode='lines',
                line=dict(color=color, width=2),
                marker=dict(color=color, size=5),
                opacity=0.75,
                name=model,
                showlegend=show_legend_
            )
        )

        show_legend_ = False

# ==================================================
# CUTOFFS VERTICAL LINES
# ==================================================
for cutoff in cutoffs:
    fig.add_vline(
        x=cutoff + pd.Timedelta(hours=3),
        line_width=1,
        line_dash="dot",
        line_color="black",
        opacity=0.5
    )
    fig.add_vline(
        x=cutoff + pd.Timedelta(hours=3 * experiment_['horizon']),
        line_width=1,
        line_dash="dot",
        line_color="black",
        opacity=0.5
    )

# ==================================================
# FINAL LAYOUT
# ==================================================
plotlymodex.main_layout(
    fig, width=1200, height=600,
    x=r"$\mathrm{time_{\ freq: 3 h}}$",
    y=r"$\mathrm{kg\ kg^{-1}}$",
    title='Pollutant Forecasting | Cross-Validation Windows'
)

fig.update_layout(
    legend_title_text='Series',
    hovermode='x unified',

    # Fonte global
    font=dict(
        family="Times New Roman",
        size=14,
        color="black"
    ),

    # Eixo X
    xaxis=dict(
        showgrid=False, zeroline=False,
        title_font=dict(family="Times New Roman"),
        tickfont=dict(family="Times New Roman")
    ),

    # Eixo Y
    yaxis=dict(
        zeroline=False,
        title_font=dict(family="Times New Roman"),
        tickfont=dict(family="Times New Roman")
    ),
)

fig.update_layout(
    # Margens bem mais compactas
    margin=dict(
        l=60,
        r=10,
        t=60, 
        b=50
    ),

    # Título mais compacto
    title=dict(
        y=0.97,
        yanchor='top'
    )
)

fig.update_layout(
    legend=dict(
        x=0.01,      # esquerda
        y=0.99,      # topo
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.7)',  # opcional
        bordercolor='black',              # opcional
        borderwidth=2                     # opcional
    )
)

fig.show()

## **Estatísticas**

In [41]:
# ==================================================
# METRICS
# ==================================================

def mae(y, yhat):
    return np.mean(np.abs(y - yhat))


def mse(y, yhat):
    return np.mean((y - yhat) ** 2)


def rmse(y, yhat):
    return np.sqrt(mse(y, yhat))


def smape(y, yhat):
    denom = (np.abs(y) + np.abs(yhat)) / 2
    mask = denom != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]) / denom[mask])


def mae_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]))


def bias_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(yhat[mask] - y[mask])


def skill_score(model_err, baseline_err):
    if baseline_err == 0 or np.isnan(baseline_err):
        return np.nan
    return 1 - model_err / baseline_err


def extreme_event_metrics(y, yhat, threshold):
    y_event = y >= threshold
    yhat_event = yhat >= threshold

    tp = np.sum(y_event & yhat_event)
    fp = np.sum(~y_event & yhat_event)
    fn = np.sum(y_event & ~yhat_event)

    precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan

    if precision > 0 and recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = np.nan

    return precision, recall, f1


# ==================================================
# EVALUATION LOOP (ROBUST & NO LEAKAGE)
# ==================================================

BASE = Path("Results/RQ1/full")
baseline_name = "Naive"

STEPS_PER_YEAR = 365 * 8  # dados 3h
WINDOW_P95 = STEPS_PER_YEAR

records = []

for file in BASE.glob("*.csv"):

    pollutant, horizon = file.stem.split("_", 1)
    print(f"Evaluating | {pollutant} | {horizon}")

    # --------------------------------------
    # LOAD PREDICTIONS
    # --------------------------------------
    df_pred = pd.read_csv(file)
    df_pred['ds'] = pd.to_datetime(df_pred['ds'])
    df_pred['cutoff'] = pd.to_datetime(df_pred['cutoff'])

    # --------------------------------------
    # LOAD FULL GROUND TRUTH
    # --------------------------------------
    df_true = pollutants_dict[pollutant]['df'].copy()
    df_true['ds'] = pd.to_datetime(df_true['ds'])

    # --------------------------------------
    # DEFINE MODEL COLUMNS (CLEAN)
    # --------------------------------------
    model_cols = [
        c for c in df_pred.columns
        if (
            c not in ["unique_id", "ds", "cutoff", "y"]
            and not c.startswith("fit_time")
            and df_pred[c].dtype in [np.float64, np.float32]
        )
    ]

    if baseline_name not in model_cols:
        raise ValueError(f"Baseline '{baseline_name}' not found.")

    # --------------------------------------
    # GROUP BY FOLD
    # --------------------------------------
    for (uid, cutoff), df_fold in df_pred.groupby(["unique_id", "cutoff"]):

        # ----------------------------------
        # TRAIN WINDOW (NO LEAKAGE)
        # ----------------------------------
        y_train_full = (
            df_true
            .query("unique_id == @uid and ds <= @cutoff")
            ['y']
            .values
        )

        if len(y_train_full) < WINDOW_P95:
            continue

        y_train_recent = y_train_full[-WINDOW_P95:]
        p95 = np.percentile(y_train_recent, 95)

        # ----------------------------------
        # TEST HORIZON
        # ----------------------------------
        y_full = df_fold['y'].values
        y_base_full = df_fold[baseline_name].values

        # baseline válido
        mask_base_valid = ~np.isnan(y_base_full)

        if mask_base_valid.sum() == 0:
            continue

        y_base_valid = y_full[mask_base_valid]
        yhat_base = y_base_full[mask_base_valid]

        mae_base = mae(y_base_valid, yhat_base)
        mae_base_p95 = mae_conditional(y_base_valid, yhat_base, p95)

        # ----------------------------------
        # EVALUATE EACH MODEL
        # ----------------------------------
        for model in model_cols:

            yhat_full = df_fold[model].values

            mask_valid = ~np.isnan(yhat_full)

            if mask_valid.sum() == 0:
                continue

            y_valid = y_full[mask_valid]
            yhat = yhat_full[mask_valid]
            y_base_valid = y_base_full[mask_valid]

            # métricas gerais
            mae_model = mae(y_valid, yhat)
            rmse_model = rmse(y_valid, yhat)
            smape_model = smape(y_valid, yhat)

            # baseline ajustado ao mesmo subconjunto
            mae_base_model = mae(y_valid, y_base_valid)
            mae_base_model_p95 = mae_conditional(y_valid, y_base_valid, p95)

            # extremos
            mae_p95 = mae_conditional(y_valid, yhat, p95)
            skill_p95 = skill_score(mae_p95, mae_base_model_p95)
            bias_p95 = bias_conditional(y_valid, yhat, p95)

            precision, recall, f1 = extreme_event_metrics(
                y_valid, yhat, p95
            )

            records.append({
                "pollutant": pollutant,
                "horizon": horizon,
                "unique_id": uid,
                "cutoff": cutoff,
                "model": model,

                # métricas médias
                "MAE": mae_model,
                "RMSE": rmse_model,
                "sMAPE": smape_model,

                # extremos
                "MAE_p95": mae_p95,
                "Skill_p95": skill_p95,
                "Bias_p95": bias_p95,
                "Precision_p95": precision,
                "Recall_p95": recall,
                "F1_p95": f1,
            })

# ==================================================
# FINAL DATAFRAME
# ==================================================

metrics_df = pd.DataFrame(records)
metrics_df.to_csv(r'..\Masters 26\Results\RQ1\metrics.csv')

Evaluating | go3 | 14days
Evaluating | go3 | 1days
Evaluating | go3 | 30days
Evaluating | go3 | 7days
Evaluating | no2 | 14days
Evaluating | no2 | 1days
Evaluating | no2 | 30days
Evaluating | no2 | 7days
Evaluating | pm10 | 14days
Evaluating | pm10 | 1days
Evaluating | pm10 | 30days
Evaluating | pm10 | 7days
Evaluating | pm2p5 | 14days
Evaluating | pm2p5 | 1days
Evaluating | pm2p5 | 30days
Evaluating | pm2p5 | 7days


In [23]:
summary = (
    metrics_df
    .groupby(["model"])
    .agg({
        "MAE": "mean",
        "RMSE": "mean",
        "sMAPE": "mean",
        "MAE_p95": "mean",
        "Skill_p95": "mean",
        "F1_p95": "mean"
    })
    .sort_values("MAE")
)
summary

,MAE,RMSE,sMAPE,MAE_p95,Skill_p95,F1_p95
model,,,,,,
NHITS,5.569817e-09,7.266401e-09,0.438836,1.470321e-08,0.095309,0.498889
NBEATS-I,5.741226e-09,7.476669e-09,0.450904,1.459445e-08,0.083515,0.477068
NBEATS-G,5.841329e-09,7.590124e-09,0.455565,1.448637e-08,0.107167,0.448308
LightGBM,6.224958e-09,7.868476e-09,0.436389,1.432538e-08,0.156120,0.486962
Theta,7.312612e-09,8.966722e-09,0.536638,1.526891e-08,0.078436,0.507742
Arima,7.336950e-09,9.107353e-09,0.581289,1.496177e-08,0.051117,0.492444
SeasonalNaive-8,7.398459e-09,9.310914e-09,0.564039,1.465981e-08,-0.009039,0.452838
SeasonalNaive-56,8.080158e-09,1.031205e-08,0.601127,1.556211e-08,-0.365250,0.323193
Naive,8.545485e-09,1.043176e-08,0.709332,1.907619e-08,0.000000,0.440968


<!-- ### 1) Universo experimental e unidade de análise

A análise considera dados de reanálise do **Copernicus Atmosphere Monitoring Service (CAMS)** em grade regular, adequados para avaliação metodológica de previsão espacial consistente [1].

* Região de estudo: células localizadas nos estados
  RS, SC, PR, SP, MS, MG, RJ e ES.
* Unidade de previsão: **célula espacial** (latitude × longitude).
* Cada célula é tratada como **uma série temporal independente**, conforme prática comum em modelos globais de forecasting [2].

---

### 2) Resolução temporal e agregação

Como o objetivo envolve horizontes de **1 a 30 dias**, os dados horários são agregados para **resolução diária**:

* `ds`: data (agregação diária do `valid_time`)
* `y`: média diária da concentração do poluente

Essa escolha reduz ruído de alta frequência e melhora a estabilidade de modelos globais [3].

---

### 3) Construção dos datasets univariados

Para cada poluente
$$ p \in {\text{PM2.5, PM10, O}_3, \text{NO}_2}, $$
é construído um dataset no formato *long* exigido pela Nixtla [4]:

* `unique_id`: identificador da célula espacial
* `ds`: tempo (diário)
* `y`: concentração do poluente (p)

Cada poluente é tratado como **experimento independente**, evitando contaminação multivariada nesta RQ [5].

---

### 4) Normalização por série

Para viabilizar o treino de **modelos globais multi-séries**, cada série é normalizada **individualmente**, com parâmetros estimados **exclusivamente no conjunto de treino** [6].

Essa etapa é necessária devido à heterogeneidade de escala entre células (urbanas, rurais, industriais), conforme discutido na literatura de modelos globais [7].

---

### 5) Split temporal (anti-vazamento)

O particionamento é feito **estritamente no tempo**, comum a todos os modelos e poluentes:

* **Treino**: período inicial
* **Validação**: período intermediário (early stopping dos modelos neurais)
* **Teste**: período final (avaliação)

Nenhuma informação do conjunto de teste é utilizada em normalização, definição de percentis ou ajuste de hiperparâmetros [8].

---

### 6) Horizontes de previsão e janela de entrada

Os modelos são avaliados separadamente para os horizontes:
$$ H \in {1, 3, 7, 14, 30} \text{ dias} $$

* Janela de entrada (`input_size`) fixa para todos os modelos
* Um modelo é treinado por horizonte (direct forecasting), evitando propagação de erro recursiva [9].

---

### 7) Modelos avaliados

#### 7.1 Baselines ingênuos (locais)

* Persistência (Naive)
* Sazonal ingênuo (lag semanal)

Esses modelos fornecem referências mínimas e são amplamente utilizados como baseline em forecasting ambiental [10].

---

#### 7.2 Modelos estatísticos clássicos (locais)

* **AutoARIMA**
* **AutoETS**

Cada modelo estatístico é ajustado **individualmente por célula**, respeitando sua formulação tradicional [11].

---

#### 7.3 Baseline moderno explicável (global)

* **LightGBM global**, via MLForecast
* Uso de lags temporais e variáveis de calendário

Modelos globais baseados em árvores são reconhecidos como baselines fortes e interpretáveis em benchmarking moderno [12].

---

#### 7.4 Modelos neurais avaliados (globais)

* **N-BEATS** [13]
* **N-HiTS** [14]

Ambos são treinados como **modelos globais univariados**, compartilhando parâmetros entre todas as séries, prática estabelecida na literatura de deep learning para séries temporais [2].

---

### 8) Treinamento e hiperparâmetros

Para garantir comparabilidade e reprodutibilidade:

* Hiperparâmetros majoritariamente **off-the-shelf**
* Ajuste preliminar limitado a um cenário piloto
* Parâmetros **congelados** para todos os poluentes e horizontes

Essa estratégia segue recomendações da literatura comparativa, priorizando robustez em detrimento de tuning agressivo [7,15].

---

### 9) Métricas de avaliação

#### 9.1 Desempenho médio

* MAE
* RMSE
* sMAPE (avaliado com cautela para valores próximos de zero)

---

#### 9.2 Avaliação orientada a extremos

Para cada célula e poluente, os limiares **p95 e p99** são definidos no conjunto de treino.

No conjunto de teste, avalia-se:

* **MAE condicional** acima de p95 e p99
* **Detecção de eventos extremos**:

  * Precision
  * Recall
  * F1-score

Essa abordagem é recomendada em estudos ambientais, onde eventos raros são os mais relevantes do ponto de vista sanitário [16].

---

### 10) Agregação espacial e comparação estatística

Para cada (poluente, horizonte, modelo):

* Métricas são calculadas **por célula**
* Resultados agregados como **mediana e intervalo interquartil (p25–p75)**

Comparações entre modelos locais e globais são feitas por **teste pareado de Wilcoxon signed-rank**, avaliando se as distribuições de erro diferem significativamente [7].

---

### 11) Critério de desempenho competitivo (RQ1)

N-BEATS e N-HiTS são considerados **competitivos** quando:

1. Superam a persistência (skill score positivo)
2. Apresentam desempenho próximo ou superior ao LightGBM global
3. Mantêm desempenho estável em eventos extremos (p95/p99)

Esse critério reflete práticas consolidadas em estudos comparativos de forecasting [7,12].

--- -->

## Referências

* Buonanno A, Caliano M, Pontecorvo A, Sforza G, Valenti M, Graditi G. Global vs. Local Models for Short-Term Electricity Demand Prediction in a Residential/Lodging Scenario. Energies. 2022 Mar 10;15(6):2037. Available from: https://doi.org/10.3390/en15062037
1. Inness A, et al. *The CAMS reanalysis of atmospheric composition*. Atmos Chem Phys. 2019.
2. Salinas D, et al. *DeepAR: Probabilistic forecasting with autoregressive recurrent networks*. Int J Forecast. 2020.
3. Hyndman RJ, Athanasopoulos G. *Forecasting: Principles and Practice*. OTexts; 2021.
4. Nixtla. *NeuralForecast documentation*. 2023.
5. Box GEP, Jenkins GM. *Time Series Analysis: Forecasting and Control*. Holden-Day; 1976.
6. Makridakis S, et al. *Statistical and Machine Learning forecasting methods*. PLoS One. 2018.
7. Bianchi FM, et al. *Local vs global models for energy forecasting*. Energies. 2022.
8. Bergmeir C, Benítez JM. *On the use of cross-validation for time series predictor evaluation*. Inf Sci. 2012.
9. Taieb SB, Hyndman RJ. *Recursive and direct multi-step forecasting*. Int J Forecast. 2014.
10. Hyndman RJ. *Benchmarks for time series forecasting*. Int J Forecast. 2006.
11. Hyndman RJ, Khandakar Y. *Automatic time series forecasting*. J Stat Softw. 2008.
12. Makridakis S, et al. *The M5 accuracy competition*. Int J Forecast. 2022.
13. Oreshkin BN, et al. *N-BEATS: Neural basis expansion analysis for time series forecasting*. ICLR. 2020.
14. Challu C, et al. *N-HiTS: Neural hierarchical interpolation for time series forecasting*. AAAI. 2023.
15. Montero-Manso P, Hyndman RJ. *Principles and algorithms for forecasting groups of time series*. Int J Forecast. 2021.
16. Dabass A, et al. *Extreme air pollution events and health impacts*. Environ Health Perspect. 2018.